# Fashion-mnist分类任务

# Fashion-mnist

[经典的MNIST数据集](http://yann.lecun.com/exdb/mnist/)包含了大量的手写数字。十几年来，来自机器学习、机器视觉、人工智能、深度学习领域的研究员们把这个数据集作为衡量算法的基准之一。你会在很多的会议，期刊的论文中发现这个数据集的身影。实际上，MNIST数据集已经成为算法作者的必测的数据集之一。有人曾调侃道：*"如果一个算法在MNIST不work, 那么它就根本没法用；而如果它在MNIST上work, 它在其他数据上也可能不work！"*
 

`Fashion-MNIST`的目的是要成为MNIST数据集的一个直接替代品。作为算法作者，你不需要修改任何的代码，就可以直接使用这个数据集。`Fashion-MNIST`的图片大小，训练、测试样本数及类别数与经典MNIST**完全相同**。

这个数据集的样子大致如下（每个类别占三行）：

![](https://github.com/zalandoresearch/fashion-mnist/raw/master/doc/img/fashion-mnist-sprite.png)


## 类别标注

在Fashion-mnist数据集中，每个训练样本都按照以下类别进行了标注：

| 标注编号 | 描述 |
| --- | --- |
| 0 | T-shirt/top（T恤）|
| 1 | Trouser（裤子）|
| 2 | Pullover（套衫）|
| 3 | Dress（裙子）|
| 4 | Coat（外套）|
| 5 | Sandal（凉鞋）|
| 6 | Shirt（汗衫）|
| 7 | Sneaker（运动鞋）|
| 8 | Bag（包）|
| 9 | Ankle boot（踝靴）|


## 任务描述


`Fashion-MNIST`是一个替代[MNIST手写数字集](http://yann.lecun.com/exdb/mnist/)的图像数据集。 它是由Zalando（一家德国的时尚科技公司）旗下的[研究部门](https://research.zalando.com/)提供。其涵盖了来自10种类别的共7万个不同商品的正面图片。Fashion-MNIST的大小、格式和训练集/测试集划分与原始的MNIST完全一致。60000/10000的训练测试数据划分，28x28的灰度图片。你可以直接用它来测试你的机器学习和深度学习算法性能，且**不需要**改动任何的代码。


本次任务需要针对`Fashion-MNIST`数据集，设计、搭建、训练机器学习模型，能够尽可能准确地分辨出测试数据地标签。

## 文档说明 


数据集文件分为训练集和测试集部分，对应文件如下：

- 训练数据：`train-images-idx3-ubyte.gz` 
- 训练标签：`train-labels-idx1-ubyte.gz`
- 测试数据：`t10k-images-idx3-ubyte.gz`


## 参考文献

[1] Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms. Han Xiao, Kashif Rasul, Roland Vollgraf. arXiv:1708.07747

[2] https://github.com/zalandoresearch/fashion-mnist/

# 评估说明

## 评价指标

本次任务采用 [ACC（Accuracy)](https://baike.baidu.com/item/%E5%87%86%E7%A1%AE%E7%8E%87) 作为模型的评价标准。

## 在线评估

评估函数首先会验证选手提交的预测结果文件是否符合要求，主要验证了以下要求:

1. 提交的预测文件是否存在重复ID
2. 提交的预测文件ID是否与测试集文件ID不匹配

通过验证后的文件会用以ACC为测评指标的函数进行计算评估。


## 文件格式

由于测评脚本已经统一，为保证脚本的顺利运行，在进行测评时，要求选手提交的`预测文件`拥有规范的字段名和字段格式，预测文件具体要求如下：

| NO | 字段名称 | 数据类型 | 字段描述 |
| -------- | -------- | -------- | -------- |
| 1    | ID     | int    | ID序列     |
| 2    | Prediction   | int     | 预测结果（类别值）   |

正确格式的提交文件样例: `submission_random.csv`。

## 基准算法

本次任务采用不同的基准算法，获得模型的ACC如下：
- 随机基准算法ACC：0.09440
- 弱基准算法ACC：0.90452

在评估时，以弱基准算法的ACC作为达标线。

## 终审评估

本次任务的终审评估将挑选在评分指标位于前10名的同学进行项目报告撰写，以描述模型、算法及实验等相关内容和结果，报告排版要求届时发布。

除此以外，为保证竞赛的公平性，进入终审评估的同学需要提交项目代码，由助教进行模型的有效性验证。

如发现实验结果有较大差异，或者模型无法复现等问题，组委会将取消营员本次14天陪你挑战《动手学深度学习》的结营资格，并且进行公示。

# 任务说明

> 针对`Fashion-MNIST`数据集，设计、搭建、训练机器学习模型，能够尽可能准确地分辨出测试数据集标签。

> 数据集文件分为训练集和测试集部分，对应文件如下：

>- 训练数据：`train-images-idx3-ubyte.gz` 
- 训练标签：`train-labels-idx1-ubyte.gz`
- 测试数据：`t10k-images-idx3-ubyte.gz`

# 代码实现

In [1]:
# 导入必需包
import os
import sys
import time
import math
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms

In [2]:
# 定义全剧平均池化类：普通的平均池化的窗口形状 --> Input：(H, W)
class GlobalAvgPool2d(nn.Module):
    """
    全局平均池化层
    可通过将普通的平均池化的窗口形状设置成输入的高和宽实现
    """
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

# 定义展平类
class FlattenLayer(torch.nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    def forward(self, x): # x shape: （N, C, H, W）
        return x.view(x.shape[0], -1)

# 定义残差类（使用1*1卷积层来修改通道数）
class Residual(nn.Module): 
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        """
            use_1×1conv: 是否使用额外的1x1卷积层来修改通道数
            stride: 卷积层的步幅
        """
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels) # 批归一化
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

In [3]:
# 定义加载数据集函数
def load_data_fashion_mnist(batch_size, root='/home/kesci/work/FashionMNIST1045', use_normalize=False, mean=None, std=None):
    """下载数据集并将其加载进内存"""

    if use_normalize: # 对输入进行归一化操作
        normalize = transforms.Normalize(mean=[mean], std=[std])
        # 进行数据增强：RandomCrop，RandomHorizontalFlip
        train_augs = transforms.Compose([transforms.RandomCrop(28, padding=2),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(), 
                    normalize])
        test_augs = transforms.Compose([transforms.ToTensor(), normalize])
    else:
        train_augs = transforms.Compose([transforms.ToTensor()])
        test_augs = transforms.Compose([transforms.ToTensor()])
    
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=train_augs)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=test_augs)
    if sys.platform.startswith('win'):
        num_workers = 0  # 0 表示不用额外的进程来加速读取数据
    else:
        num_workers = 4  # 通过参数num_workers来设置进程读取数据（使用多进程来加速数据读取）
    # DataLoader：允许使用多进程来加速数据读取
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return train_iter, test_iter

In [4]:
# 定义残差块函数
def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    '''
    resnet
    resnet使用步长为2（stride = 2）的卷积来替代pooling的作用！！！
    resnet block
    num_residuals: 当前block包含多少个残差块
    first_block: 是否为第一个block
    一个resnet block由num_residuals个残差块组成
    其中第一个残差块起到了通道数的转换和pooling的作用
    后面的若干残差块就是完成正常的特征提取
    '''
    if first_block:
        assert in_channels == out_channels # 第一个模块的输出通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

# 定义resnet模型结构
# resnet使用步长为2的卷积来替代pooling的作用！！！
net = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),   # 缩小感受野, 缩小通道数
        nn.BatchNorm2d(32),
        nn.ReLU())
        #nn.ReLU(),
        #nn.MaxPool2d(kernel_size=2, stride=2))   # 去掉maxpool缩小感受野

# 追加连续4个block
net.add_module("resnet_block1", resnet_block(32, 32, 2, first_block=True))   # channel统一减半
net.add_module("resnet_block2", resnet_block(32, 64, 2))
net.add_module("resnet_block3", resnet_block(64, 128, 2))
net.add_module("resnet_block4", resnet_block(128, 256, 2))
# 全局平均池化
net.add_module("global_avg_pool", GlobalAvgPool2d()) 
# 全连接层
net.add_module("fc", nn.Sequential(FlattenLayer(), nn.Linear(256, 10)))

print('----------------打印网络结构----------------')
print(net)

----------------打印网络结构----------------
Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (resnet_block1): Sequential(
    (0): Residual(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Residual(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )

In [5]:
print('打印 1*1*28*28 输入经过每个模块后的shape')
X = torch.rand((1, 1, 28, 28))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

打印 1*1*28*28 输入经过每个模块后的shape
0  output shape:	 torch.Size([1, 32, 28, 28])
1  output shape:	 torch.Size([1, 32, 28, 28])
2  output shape:	 torch.Size([1, 32, 28, 28])
resnet_block1  output shape:	 torch.Size([1, 32, 28, 28])
resnet_block2  output shape:	 torch.Size([1, 64, 14, 14])
resnet_block3  output shape:	 torch.Size([1, 128, 7, 7])
resnet_block4  output shape:	 torch.Size([1, 256, 4, 4])
global_avg_pool  output shape:	 torch.Size([1, 256, 1, 1])
fc  output shape:	 torch.Size([1, 10])


In [6]:
print('----------------计算数据集的均值和标准差----------------')
batch_size = 64  
train_iter, test_iter = load_data_fashion_mnist(batch_size, 
                        root='/home/kesci/work/FashionMNIST1045', use_normalize=False)
# 求整个数据集的均值
temp_sum = 0
cnt = 0
for X, y in train_iter:
    if y.shape[0] != batch_size:
        break   # 最后一个batch不足batch_size，忽略
    channel_mean = torch.mean(X, dim=(0,2,3))  # 按channel求均值(不过这里只有1个channel)
    cnt += 1   # cnt记录的是batch的个数，不是图像
    temp_sum += channel_mean[0].item()
dataset_global_mean = temp_sum / cnt
print('----------------打印整个数据集的像素均值:{}----------------'.format(dataset_global_mean))
# 求整个数据集的标准差
cnt = 0
temp_sum = 0
for X, y in train_iter:
    if y.shape[0] != batch_size:
        break   # 最后一个batch不足batch_size,这里就忽略了
    residual = (X - dataset_global_mean) ** 2
    channel_var_mean = torch.mean(residual, dim=(0,2,3))  
    cnt += 1   # cnt记录的是batch的个数，不是图像
    temp_sum += math.sqrt(channel_var_mean[0].item())
dataset_global_std = temp_sum / cnt
print('----------------打印整个数据集的像素标准差:{}----------------'.format(dataset_global_std))               

----------------计算数据集的均值和标准差----------------
----------------打印整个数据集的像素均值:0.286041207221936----------------
----------------打印整个数据集的像素标准差:0.35289938988137576----------------


In [7]:
# 重新获取应用了归一化的数据集迭代器
batch_size = 64  
train_iter, test_iter = load_data_fashion_mnist(batch_size, root='/home/kesci/work/FashionMNIST1045', use_normalize=True,
                        mean = dataset_global_mean, std = dataset_global_std)

In [8]:
# 定义评估准确率
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device
    net.eval() 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            n += y.shape[0]
    net.train() # 改回训练模式
    return acc_sum / n

In [9]:
# 定义训练模型
def train_model(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print("----------------training on----------------", device)
    loss = torch.nn.CrossEntropyLoss()
    best_test_acc = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.4f, test acc %.4f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))
        if test_acc > best_test_acc:
            print('find best! save at best.pth') # model/
            best_test_acc = test_acc
            torch.save(net.state_dict(), 'best.pth') # model/
            #utils.save_model({
            #    'arch': args.model,
            #    'state_dict': net.state_dict()
            #}, 'saved-models/{}-run-{}.pth.tar'.format(args.model, run))

In [10]:
print('----------------******训练******----------------')
lr, num_epochs = 0.01, 10
# optimizer = optim.Adam(net.parameters(), lr=lr) Adam优化算法
# 修改优化算法 （区分Adam与SGD的异同）
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4) 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_model(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

----------------******训练******----------------
----------------training on---------------- cpu
epoch 1, loss 0.5268, train acc 0.8025, test acc 0.8496, time 590.7 sec
find best! save at best.pth
epoch 2, loss 0.3248, train acc 0.8821, test acc 0.8923, time 598.0 sec
find best! save at best.pth
epoch 3, loss 0.2795, train acc 0.8965, test acc 0.9000, time 591.3 sec
find best! save at best.pth
epoch 4, loss 0.2511, train acc 0.9076, test acc 0.9095, time 592.0 sec
find best! save at best.pth
epoch 5, loss 0.2334, train acc 0.9150, test acc 0.9145, time 598.9 sec
find best! save at best.pth
epoch 6, loss 0.2211, train acc 0.9183, test acc 0.9177, time 591.1 sec
find best! save at best.pth
epoch 7, loss 0.2125, train acc 0.9215, test acc 0.9158, time 593.8 sec
epoch 8, loss 0.2034, train acc 0.9258, test acc 0.9193, time 595.5 sec
find best! save at best.pth
epoch 9, loss 0.1977, train acc 0.9278, test acc 0.9216, time 591.7 sec
find best! save at best.pth
epoch 10, loss 0.1897, train acc 

In [11]:
print('----------------加载最优模型----------------')
net.load_state_dict(torch.load('best.pth')) # model/
net = net.to(device)

print('----------------******测试******----------------')
net.eval() 
id = 0
preds_list = []
with torch.no_grad():
    for X, y in test_iter:
        batch_pred = list(net(X.to(device)).argmax(dim=1).cpu().numpy())
        for y_pred in batch_pred:
            preds_list.append((id, y_pred))
            id += 1

----------------加载最优模型----------------
----------------******测试******----------------


In [12]:
print('----------------Submission Result----------------')
with open('submission.csv', 'w') as f:
    f.write('ID,Prediction\n')
    for id, pred in preds_list:
        f.write('{},{}\n'.format(id, pred))

----------------Submission Result----------------
